In [162]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [163]:
df = pd.read_csv('5310_org_info_manualfill.csv')

In [164]:
df.sample(5)

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
97,SMOOTH,NaN,NaN,Santa Maria Organization of Transportation Hel...,NaN,NaN,Company,NaN,Santa Maria,Santa Barbara,"Guadalupe Flyer,Guadalupe Shuttle,Santa Maria/...","Manual Match, Airtable"
93,On Lok Senior Health Services,NaN,NaN,On Lok,NaN,NaN,Non-Profit Organization,NaN,San Francisco,San Francisco,On Lok Senior Health Services,"Manual Match, Airtable"
63,Amador Transit,9R02-91000,11.0,Amador Regional Transit System,NaN,ARTS,Independent Public Agency or Authority of Tran...,Rural Reporter,Jackson,Amador,"Amador Transit,Amador Transit Dial-A-Ride","Manual Match, NTD"
15,Desert ARC,NaN,NaN,Desert ARC,NaN,NaN,Non-Profit Organization,NaN,Palm Desert,Riverside,Desert ARC,Name Match; Airtable
6,Catholic Charities of the Diocese of Stockton,NaN,408.0,Catholic Charities of the Diocese of Stockton,NaN,NaN,Non-Profit Organization,NaN,Stockton,San Joaquin,Catholic Charities Stockton Senior Transportation,Name Match; Airtable


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   organization_name           147 non-null    object 
 1   ntd_id                      37 non-null     object 
 2   itp_id                      78 non-null     float64
 3   name_NTD_Airtable           108 non-null    object 
 4   doing_business_as           7 non-null      object 
 5   reporter_acronym            28 non-null     object 
 6   organization_type           147 non-null    object 
 7   reporter_type               37 non-null     object 
 8   city                        146 non-null    object 
 9   county                      146 non-null    object 
 10  mobility_services_operated  90 non-null     object 
 11  merge_status                147 non-null    object 
dtypes: float64(1), object(11)
memory usage: 13.9+ KB


In [166]:
df>>count(_.county)>>arrange(-_.n)

,county,n
32,Riverside,10
0,Alameda,9
35,San Bernardino,9
9,Contra Costa,8
38,San Francisco,7
19,Kern,6
24,Mendocino,5
51,Solano,5
33,Sacramento,4
37,San Diego,4


In [167]:
df>>filter(_.county.str.contains(", "))

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
2,Area 1 Agency on Aging,NaN,399.0,Area 1 Agency on Aging,NaN,NaN,Non-Profit Organization,NaN,Eureka,"Humboldt, De Norte",NaN,Name Match; Airtable
3,Big Valley 50 Plus,NaN,404.0,Big Valley 50 Plus,NaN,NaN,Non-Profit Organization,NaN,Beiber,"Lassen, Modoc",Big Valley 50 Plus,Name Match; Airtable
13,Common Ground Senior Services,NaN,419.0,Common Ground Senior Services,NaN,NaN,Non-Profit Organization,NaN,San Andreas,"Calaveras, Amador, Tuolumne",Silver Streak Transport,Name Match; Airtable
19,Friends of Children with Special Needs,NaN,NaN,Friends of Children with Special Needs,NaN,NaN,Non-Profit Organization,NaN,Fremont,"Santa Clara, Alameda",NaN,Name Match; Airtable
24,Inyo-Mono Association for the Handicapped,NaN,434.0,Inyo-Mono Association for the Handicapped,NaN,NaN,Non-Profit Organization,NaN,Bishop,"Inyo, Mono",Inyo-Mono Association for the Handicapped,Name Match; Airtable
51,Self-Help for the Elderly,NaN,NaN,Self-Help for the Elderly,NaN,NaN,Non-Profit Organization,NaN,San Francisco,"San Francisco, Alameda, San Mateo, Santa Clara",Self-Help for the Elderly,Name Match; Airtable
52,Tahoe Transportation District,91092,331.0,Tahoe Transportation District,NaN,TTD,Independent Public Agency or Authority of Tran...,Full Reporter,Zephyr Cove,"Douglas, Placer, Washoe, Carson City","Tahoe Truckee Area Regional Transportation,Tah...",Name Match; Airtable
53,The Arc of Amador and Calaveras,NaN,461.0,The Arc of Amador and Calaveras,NaN,NaN,Non-Profit Organization,NaN,Sutter Creek,"Amador, Calaveras",The Arc of Amador and Calaveras,Name Match; Airtable
74,Community Bridges Liftline,NaN,420.0,Community Bridges Inc,NaN,NaN,Non-Profit Organization,NaN,Watsonville,"Monterey, San Mateo, San Benito, Santa Clara, ...",Lift Line,"Manual Match, Airtable"
80,Lutheran Social Services of Southern California,NaN,439.0,Lutheran Social Services,NaN,NaN,Non-Profit Organization,NaN,Orange,"Orange, Los Angeles, Riverside, San Bernardino...",NaN,"Manual Match, Airtable"


In [168]:
df>>count(_.organization_type)>>arrange(-_.n)

,organization_type,n
5,Non-Profit Organization,90
3,Independent Public Agency or Authority of Tran...,28
0,"City, County or Local Government Unit or Depar...",12
2,Independent Agency,5
1,Company,4
4,MPO/RTPA,3
6,Other,3
7,Private-Non-Profit Corporation,1
8,University,1


## Charts

In [169]:
df.sample()

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
52,Tahoe Transportation District,91092,331.0,Tahoe Transportation District,NaN,TTD,Independent Public Agency or Authority of Tran...,Full Reporter,Zephyr Cove,"Douglas, Placer, Washoe, Carson City","Tahoe Truckee Area Regional Transportation,Tah...",Name Match; Airtable


In [170]:
def labeling(word):
    LABEL_DICT = {
        "dist": "District",
        "nunique": "Number of Unique",
        "n":"Count",
        "name_NTD_Airtable":"Organization Name"
    }

    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace("n_", "Number of ").title()
        word = word.replace("unique_", "Number of Unique ").title()
        word = word.replace("_", " ").title()

    return word

In [171]:
def bar_chart(df, x_col, y_col, color_col, chart_title=""):

    bar = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
            )
        ).properties(title=chart_title))
    
    chart = styleguide.preset_chart_config(bar)
    
    return chart

In [172]:
org_counts = df>>count(_.organization_type)

In [173]:
org_counts

,organization_type,n
0,"City, County or Local Government Unit or Depar...",12
1,Company,4
2,Independent Agency,5
3,Independent Public Agency or Authority of Tran...,28
4,MPO/RTPA,3
5,Non-Profit Organization,90
6,Other,3
7,Private-Non-Profit Corporation,1
8,University,1


In [174]:
bar_chart(org_counts,
    "organization_type",
    "n",
    "organization_type",
    "5310 Organization Types"
)

alt.Chart(...)

In [175]:
def charttext(df, x_col, y_col, color_col):
    bars = alt.Chart(df).mark_bar().encode(
            x=alt.X(x_col, title=labeling(x_col), sort=("-y")),
            y=alt.Y(y_col, title=labeling(y_col)),
            color=alt.Color(
                color_col,
                scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
    ))

    text = bars.mark_text(
        align="left", baseline="middle", color="black", dy=3
    ).encode(
        text= y_col, color=alt.value("black")
    )
    
    display(styleguide.preset_chart_config(bars + text))

In [176]:
charttext(org_counts, "organization_type", "n", "organization_type")

alt.LayerChart(...)

In [177]:
## create long df for county counts

In [178]:
df.sample(5)

,organization_name,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
14,Desert Access and Mobility Inc,NaN,NaN,Desert Access and Mobility Inc,NaN,NaN,Non-Profit Organization,NaN,Palm Springs,Riverside,NaN,Name Match; Airtable
47,San Joaquin Regional Transit District,90012,284.0,San Joaquin Regional Transit District,NaN,RTD,Independent Public Agency or Authority of Tran...,Full Reporter,Stockton,San Joaquin,"SJRTD Local,SJRTD BRT Express,SJRTD Commuter,S...",Name Match; NTD
51,Self-Help for the Elderly,NaN,NaN,Self-Help for the Elderly,NaN,NaN,Non-Profit Organization,NaN,San Francisco,"San Francisco, Alameda, San Mateo, Santa Clara",Self-Help for the Elderly,Name Match; Airtable
135,Tuolumne County Transit Agency (TCTA),9R02-91057,NaN,Tuolumne County Transit,NaN,TCT,Independent Public Agency or Authority of Tran...,Rural Reporter,Sonora,Tuolumne,NaN,"Manual Match, NTD"
125,Inland Empire United Way,NaN,NaN,NaN,NaN,NaN,Non-Profit Organization,NaN,Rancho Cucamonga,San Bernardino,NaN,No Match


In [179]:
single_county = df[~df["county"].str.contains(",", na=False)]

In [180]:
single_county>>count(_.county)

,county,n
0,Alameda,9
1,Amador,1
2,Butte,2
3,Calaveras,1
4,Colusa,1
5,Contra Costa,8
6,Del Norte,1
7,El Dorado,1
8,Humboldt,3
9,Imperial,2


In [181]:
bar_chart((single_county>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations (Single County)"
)

alt.Chart(...)

In [182]:
df1 = df>>select(_.organization_name, _.ntd_id, _.itp_id, _.organization_type, _.city, _.county, _.merge_status)

In [183]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   organization_name  147 non-null    object 
 1   ntd_id             37 non-null     object 
 2   itp_id             78 non-null     float64
 3   organization_type  147 non-null    object 
 4   city               146 non-null    object 
 5   county             146 non-null    object 
 6   merge_status       147 non-null    object 
dtypes: float64(1), object(6)
memory usage: 8.2+ KB


In [184]:
df1.explode('county')

,organization_name,ntd_id,itp_id,organization_type,city,county,merge_status
0,Angel View Inc,NaN,NaN,Non-Profit Organization,Cathedral City,Riverside,Name Match; Airtable
1,ARC Imperial Valley,NaN,398.0,Non-Profit Organization,El Centro,Imperial,Name Match; Airtable
2,Area 1 Agency on Aging,NaN,399.0,Non-Profit Organization,Eureka,"Humboldt, De Norte",Name Match; Airtable
3,Big Valley 50 Plus,NaN,404.0,Non-Profit Organization,Beiber,"Lassen, Modoc",Name Match; Airtable
4,Care-A-Van Transit Systems Inc,NaN,407.0,Company,Hemet,Riverside,Name Match; Airtable
...,...,...,...,...,...,...,...
142,Loma Linda University Health,NaN,NaN,Other,Loma Linda,San Bernardino,No Match
143,New Advances for People with Disabilities,NaN,NaN,Non-Profit Organization,Bakersfield,Kern,No Match
144,Pomeroy Recreation & Rehabilitation Center,NaN,NaN,Non-Profit Organization,San Francisco,San Francisco,No Match
145,Regents of the University of California Santa ...,NaN,NaN,University,Santa Cruz,Santa Cruz,No Match


In [185]:
## from https://medium.com/swlh/three-routes-convert-comma-separated-column-to-rows-c17c85079ecf

df1 = df1.assign(county_split = df['county'].str.split(', ')).drop("county", axis=1)\
    .rename(columns = {"county_split": "county"})


In [186]:
df1

,organization_name,ntd_id,itp_id,organization_type,city,merge_status,county
0,Angel View Inc,NaN,NaN,Non-Profit Organization,Cathedral City,Name Match; Airtable,[Riverside]
1,ARC Imperial Valley,NaN,398.0,Non-Profit Organization,El Centro,Name Match; Airtable,[Imperial]
2,Area 1 Agency on Aging,NaN,399.0,Non-Profit Organization,Eureka,Name Match; Airtable,"[Humboldt, De Norte]"
3,Big Valley 50 Plus,NaN,404.0,Non-Profit Organization,Beiber,Name Match; Airtable,"[Lassen, Modoc]"
4,Care-A-Van Transit Systems Inc,NaN,407.0,Company,Hemet,Name Match; Airtable,[Riverside]
...,...,...,...,...,...,...,...
142,Loma Linda University Health,NaN,NaN,Other,Loma Linda,No Match,[San Bernardino]
143,New Advances for People with Disabilities,NaN,NaN,Non-Profit Organization,Bakersfield,No Match,[Kern]
144,Pomeroy Recreation & Rehabilitation Center,NaN,NaN,Non-Profit Organization,San Francisco,No Match,[San Francisco]
145,Regents of the University of California Santa ...,NaN,NaN,University,Santa Cruz,No Match,[Santa Cruz]


In [187]:
df1 = df1.explode('county')

In [188]:
df1>>count(_.county)>>arrange(-_.n)

,county,n
0,Alameda,12
29,Riverside,12
32,San Bernardino,11
6,Contra Costa,10
34,San Francisco,10
39,Santa Clara,9
15,Kern,6
37,San Mateo,6
43,Solano,6
21,Mendocino,5


In [191]:
bar_chart((df1>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations Operating"
)

alt.Chart(...)